# Intro au caching - Redis

Les caches sont des instances qui permettent de retenir ponctuellement des paires clef/valeur lorsque l'obtention des valeurs est couteuse/longue. Voyons les caches comme des dictionnaires / hashmap qui permettent une récupération très rapides de ces valeur indexées par une clef. Le stockage du dictionnaire se fait souvent en RAM pour accélérer l'insertion et la récupération.

## Principe de fonctionnement

Un cache est un intermédiaire entre 2 services : un `client` et un `serveur`. Plutôt que d'interroger directement le `serveur` pour obtenir une réponse à propos d'un objet (quelconque) `A`, le `client` va interroger le cache pour savoir s'il connait une clef qui correspond à `A` : `KEY:A`. Alors :
- si le cache ne connaît pas la clef `KEY:A`, alors
  - le `serveur` est tout de même interroger et renvoie la réponse `RESP(A)`
  - le cache intercepte `RESP(A)` et crée une entrée `KEY:A -> RESP(A)` dans son dictionnaire
  - le cache renvoie `RESP(A)` au client
- si le cache connaît `KEY:A`, alors il renvoie `RESP(A)` au `client` sans interroger le serveur

2 cas de figure peuvent justifier le recours à un cache : économiser de l'argent ou du temps.

### Caching pour économiser de l'argent
Supposons qu'un service distant doive être appelé par une notre archi logicielle. Supposons que 
- ce service ait un coût ; soit car il nécessite de payer un service tiers à l'appel (exemple: ChatGPT), soit parce que le volume d'appel engendré implique un agrandissement des ressources pour ce service
- sa réponse varie peu dans le temps (ie pas un flux vidéo, pas une donnée météo, etc...). Exemple de service distant : un service IA d'embedding de texte en vu d'une recherche vectorielle.

Notre architecture peut être amenée à évoluer, des techno peuvent changer, des data re-traitées ... Sans pour autant que les données changent fondamentalement. Dans ce cas, il est préférable d'éviter de payer des appels inutiles au service coûteux s'il faut regénérer ses outputs. Le cache intervient ici en stockant les retours des appels au service tiers une fois pour toute. Le coût d'appel est donc remplacé par un coût de stockage.

### Caching pour économiser du temps
Supposons qu'une donnée soit stockée en DB, tel le résultat d'une jointure. Supposons que plusieurs services de notre architecture nécessitent ponctuellement la connaissance de cette information. Exemple : connaître les 10 dernières commandes d'un acheteur sur un site e-commerce => les services ayant simultanément besoin de cette info : le front pour affichage, un service ML pour calcul de la probabilité d'achat du client, un service comptable de vérification que toutes les factures sont honorées ... le tout en l'espace de 5 secondes.

Chaque appel à la DB a un coût ; diviser la charge de la DB par 2 pourrait permettre de diviser les CPU et la RAM loués par ~2 (estimation grosse maille). Ainsi, la stratégie suivante :
- n'appeler la DB qu'une seule fois lorsque le premier service demande l'info
- stocker sa réponse dans un cache
- économiser les appels DB suivants en distribuant simplement à la réponse cachée aux autres services lorsqu'ils demandent l'info
=> Permet de diviser sensiblement la charge sur la DB.

## Time to live

Un cache peut programmer l'expiration des clefs enregistrées au bout de `x` secondes. Ce délai s'appelle *time to live* ou TTL. Le TTL permet d'établir un compromis entre 2 types de coûts
- coût de stockage RAM du cache qui augmente sans fin
- coût de rappel ponctuel du service caché

Lorsqu'une clef expire, le cache la supprime. Le prochain appel sur cette clef provoquera une interrogation du serveur et permettra de rafraîchir la clef dans le cache.

Le TTL permet également de faire expirer une information lorsqu'on sait qu'elle peut périmer après un certain temps. Exemple : dans un moteur de recherche web (Qwant, Google), les recherches "facebook", "youtube", "gmail" sont faites des milliers de fois par seconde. Or il inutile de déclencher la cascade de calcul qui permet d'y répondre à chaque fois - les résultats attendus risquent de ne changer qu'en quelques heures. Ainsi, programmer un TTL de 3600 secondes permet :
- d'économiser 3600*1000=3.6M requêtes au moteur
- de maintenir les résultats à jour à intervalle régulier


## Redis
Redis, pour *RE*mote *DI*ctionary *S*erver, est un cache très répandu que nous allons aborder ici.

In [1]:
import redis
import requests
import time

# Redis connection details
REDIS_HOST = "redis"  # Use "localhost" if running Redis locally outside of Docker
REDIS_PORT = 6379
CACHE_EXPIRATION = 300

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 5.1 MB/s eta 0:00:00a 0:00:01
